# Project Luther

The goal of project luther is to create a linear regression model for supervised machine learning. We also need to incorporate web scrapping into the project to get practice finding what we need.  The zillow screen is pretty much a table that links to a new site where I can pull the data, so it will be a 2 stage request. One for the page, the other for the list of urls I get to the actual pages.  I can also pull out the links to the next pagination of the table, and pull it that way.  Zillows urls are set up for when you search for a zip code, it looks like https://www.zillow.com/homes/98116_rb/.  

Packages needed for this project:
 * Pandas
 * Numpy
 * BeautifulSoup
 * Requests
 * Regex

In [114]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
import re
import time
import csv

In [457]:
#Proxies need to be changed each run until data is collected!
proxies = {
    "http": 'http://47.206.51.67:8080', 
    "https": 'http://47.206.51.67:8080'
}
prox2 = {
    "http": 'http://45.32.193.119:8118', 
    "https": 'http://45.32.193.119:8118'
}
prox3 = {
    "http": 'http://52.207.93.207:80', 
    "https": 'http://52.207.93.207:80'
}
prox4 = {
    "http": 'http://159.65.110.167:3128', 
    "https": 'http://159.65.110.167:3128'
}
base = 'https://www.ted.com'
start_url = 'https://www.ted.com/talks'

response = rq.get(start_url)

In [12]:
print(response.text)

<!DOCTYPE html>
<!--[if lt IE 8]> <html class="no-js loggedout oldie ie7" lang="en"> <![endif]-->
<!--[if IE 8]> <html class="no-js loggedout oldie ie8" lang="en"> <![endif]-->
<!--[if gt IE 8]><!--><html class='no-js loggedout' lang='en'><!--<![endif]-->
<head>
<script>
  (function (H){
  H.className=H.className.replace(/\bno-js\b/,'js');
  if (('; '+document.cookie).match(/; _ted_user_id=/)) H.className=H.className.replace(/\bloggedout\b/,'loggedin');
  })(document.documentElement)
</script><meta charset='utf-8'>
<title>TED Talks</title>
<meta name="description" content="TED Talks are influential videos from expert speakers on education, business, science, tech and creativity, with subtitles in 100+ languages. Ideas free to stream and download." />
<meta name="rss-feed" content="https://www.ted.com/feeds/talks.rss" />
<meta name="keywords" content="TED, Talks, Themes, Speakers, Technology, Entertainment, Design" />
<link rel="mask-icon" href="https://pa.tedcdn.com/mask-icon.svg" colo

In [22]:
page = response.text
mainpage = soup(page,"lxml")

In [191]:
talks = re.compile('/talks/')
views = re.compile("Related Tags")

In [367]:
def GetVid(mainlist):
    vid_str = []
    l = len(mainlist)-1
    for i in range(0,l,2):
        new_str = str(mainlist[i])
        vid_str.append(new_str.split('"')[5])
    return vid_str

In [97]:
#Getting all cases where we have a /talks url
list_1 = mainpage.find_all(href=talks)
list_1[0]

<a class=" ga-link" data-ga-context="talks" href="/talks/yasin_kakande_what_s_missing_in_the_global_debate_over_refugees">
<span class="thumb thumb--video thumb--crop-top"><span class="thumb__sizer"><span class="thumb__tugger"><img alt="" class=" thumb__image" crop="top" play="267" src="https://pi.tedcdn.com/r/pe.tedcdn.com/images/ted/7237ba9d497df4cce4e706e9747770d9c17ada46_2880x1620.jpg?quality=89&amp;w=320"/><span class="thumb__aligner"></span></span></span><span class="thumb__duration"> 4:27</span></span>
<span class="talk-link__image__message">
Under 6 minutes
</span>
</a>

In [98]:
lnk_list = GetVid(list_1)
lnk_list

['/talks/yasin_kakande_what_s_missing_in_the_global_debate_over_refugees',
 '/talks/robin_steinberg_what_if_we_ended_the_injustice_of_bail',
 '/talks/malika_whitley_how_the_arts_help_homeless_youth_heal_and_build',
 '/talks/lera_boroditsky_how_language_shapes_the_way_we_think',
 '/talks/jose_andres_how_a_team_of_chefs_fed_puerto_rico_after_hurricane_maria',
 '/talks/tara_houska_the_standing_rock_resistance_and_our_fight_for_indigenous_rights',
 '/talks/kasiva_mutua_how_i_use_the_drum_to_tell_my_story',
 '/talks/danny_hillis_exploring_options_for_solar_geoengineering',
 '/talks/andrew_dent_to_eliminate_waste_we_need_to_rediscover_thrift',
 '/talks/drew_philp_my_500_house_in_detroit_and_the_neighbors_who_helped_me_rebuild_it',
 '/talks/irina_kareva_math_can_help_uncover_cancer_s_secrets',
 '/talks/raphael_arar_how_we_can_teach_computers_to_make_sense_of_our_emotions',
 '/talks/christian_picciolini_my_descent_into_america_s_neo_nazi_movement_and_how_i_got_out',
 '/talks/judith_heumann_our

## Creating a list for all ted talks

The ted talks pagination has teh url /talks?page=3, therefore we can simplify this to a 'talks?page=' + a list of numbers from 2 to 77 in '77'.  Then we call the 

In [127]:
page_lst = [str(i) for i in range(2,78)]
mid = '/talks?page='

In [361]:
url_list = [base+mid+i for i in page_lst]
url_list.insert(0,'https://www.ted.com/talks')

['https://www.ted.com/talks', 'https://www.ted.com/talks?page=2', 'https://www.ted.com/talks?page=3', 'https://www.ted.com/talks?page=4', 'https://www.ted.com/talks?page=5', 'https://www.ted.com/talks?page=6', 'https://www.ted.com/talks?page=7', 'https://www.ted.com/talks?page=8', 'https://www.ted.com/talks?page=9', 'https://www.ted.com/talks?page=10', 'https://www.ted.com/talks?page=11', 'https://www.ted.com/talks?page=12', 'https://www.ted.com/talks?page=13', 'https://www.ted.com/talks?page=14', 'https://www.ted.com/talks?page=15', 'https://www.ted.com/talks?page=16', 'https://www.ted.com/talks?page=17', 'https://www.ted.com/talks?page=18', 'https://www.ted.com/talks?page=19', 'https://www.ted.com/talks?page=20', 'https://www.ted.com/talks?page=21', 'https://www.ted.com/talks?page=22', 'https://www.ted.com/talks?page=23', 'https://www.ted.com/talks?page=24', 'https://www.ted.com/talks?page=25', 'https://www.ted.com/talks?page=26', 'https://www.ted.com/talks?page=27', 'https://www.ted

In [368]:
def GetVidLst(vurl, prox = None):
    resp = rq.get(vurl, prox)
    ppage = resp.text
    vidpage = soup(ppage,"lxml")
    vlist = vidpage.find_all(href=talks)
    vid_lnk_list = GetVid(vlist)
    return vid_lnk_list

In [369]:
#Creating a list of all the ted vides
fin = []
count=0
proxy = None
for i in url_list:
    count+=1
    fin +=GetVidLst(i, proxy)
    time.sleep(5)
    if count == 30 or count == 60 or count == 90 or count == 120:
        time.sleep(10)
        if count ==30:proxy = proxies
        if count ==60: proxy = prox2
        if count ==90: proxy = prox3
        if count ==120: 
            proxy = None 
            count = 10

In [322]:
fin

'/talks/leo_igwe_why_i_choose_humanism_over_faith'

## Create Ted Talk Dictionary

Now that we have our list of all the videos, we are going to create a dictionary with the following format:
{'Title':\['Views', \['Tags'\], Date Posted\]}.  

To accomplish this, I first did all the work on a single page, then I will create functions for each step and call them in a for loop over the list 'fin'.

In [373]:
#Changes views string to an integer
def view_to_int(viewstring):
    viewstring= viewstring.replace('\n', '').replace(',', '')
    return int(viewstring)

In [211]:
curl = base+fin[0]
resp2 = rq.get(curl)
rest = resp2.text
site_txt = soup(rest,"lxml")


In [459]:
def get_Views(site_text):
    views_list = []
    viewint=0
    views = site_text.find_all(class_=" d:n f-w:700 f:.9 f:1@xxl c:white ")
    str_list2 = str(views).split('<span')[1].split('\t')
    viewint = view_to_int(str_list2[5])
    return viewint

In [428]:
#Cleaning up site text to get tags
def Cleanup(site_text):
    super_comment = site_text.find_all(text = re.compile('tag'))
    the_max = max(super_comment, key=len)
    super_comment=the_max.split('tags')
    super_comment = super_comment[1].split("\"")
    tag=[]
    tag =[super_comment[i] for i in range(2,9,2)]
    return tag

In [377]:

tag_list= Cleanup(site_txt)
tag_list

['TED Fellows', 'government', 'history', 'humanity']

In [307]:
def getDate(site_text):
    super_comment = str(site_text.find_all(text = re.compile('recorded_at')))
    inter = re.findall(re.compile('"recorded_at"'), super_comment)
    intermed = super_comment.split(inter[0])
    intermed = intermed[1].split(':')
    date = intermed[1].replace('T00', '').replace('"', "")
    return date

In [460]:
#Creating a list of all the ted vides
data = {}
count=0
tot_views=[]
proxy = prox3
for i in fin:
    time.sleep(5)
    count+=1
    curl = base+i
    response = rq.get(curl, proxy)
    web_page = response.text
    webpage = soup(web_page,"lxml")
    date = getDate(webpage)
    tag_list = Cleanup(webpage)
    tot_views = get_Views(webpage)
    data[i] = [date,tot_views,tag_list]
    if count == 25 or count == 50 or count == 75 or count == 100:
        time.sleep(10)
        if count ==25:proxy = proxies
        if count ==50: proxy = prox2
        if count ==75: proxy = prox4
        if count ==100: 
            proxy = prox3
            count = 0



IndexError: list index out of range

In [462]:
len(data)

{'/talks/adam_alter_why_our_screens_make_us_less_happy': ['2017-04-24',
  2915234,
  ['Addiction', 'computers', 'interface design', 'psychology']],
 '/talks/adong_judith_how_i_use_art_to_bridge_misunderstanding': ['2017-08-27',
  540027,
  ['Africa', 'TED Fellows', 'activism', 'art']],
 '/talks/alastair_gray_how_fake_handbags_fund_terrorism_and_organized_crime': ['2017-11-14',
  1127728,
  ['business', 'corruption', 'crime', 'terrorism']],
 '/talks/alexander_wagner_what_really_motivates_people_to_be_honest_in_business': ['2016-11-11',
  1336721,
  ['TEDx', 'business', 'corruption', 'economics']],
 '/talks/alexis_charpentier_how_record_collectors_find_lost_music_and_preserve_our_cultural_heritage': ['2016-11-12',
  752230,
  ['TEDx', 'art', 'culture', 'music']],
 '/talks/alvin_irby_how_to_inspire_every_child_to_be_a_lifelong_reader': ['2017-11-28',
  774478,
  ['TED Residency', 'children', 'education', 'identity']],
 '/talks/amar_inamdar_the_thrilling_potential_for_off_grid_solar_energy

In [463]:
#Writing a pickle file
import pickle

with open('my_data.pkl', 'wb') as picklefile:
    pickle.dump(data, picklefile)

In [464]:
#reading a pickle file
with open("my_data.pkl", 'rb') as picklefile: 
    my_old_data = pickle.load(picklefile)

## To Pandas

Now that we have our dictionary of values, we are changing it to a pandas dataframe for easier manipulation.

There will then be some prelimnary exploring to see if it is formatted correctly or if more manipulation is needed.

In [465]:
my_old_data

{'/talks/adam_alter_why_our_screens_make_us_less_happy': ['2017-04-24',
  2915234,
  ['Addiction', 'computers', 'interface design', 'psychology']],
 '/talks/adong_judith_how_i_use_art_to_bridge_misunderstanding': ['2017-08-27',
  540027,
  ['Africa', 'TED Fellows', 'activism', 'art']],
 '/talks/alastair_gray_how_fake_handbags_fund_terrorism_and_organized_crime': ['2017-11-14',
  1127728,
  ['business', 'corruption', 'crime', 'terrorism']],
 '/talks/alexander_wagner_what_really_motivates_people_to_be_honest_in_business': ['2016-11-11',
  1336721,
  ['TEDx', 'business', 'corruption', 'economics']],
 '/talks/alexis_charpentier_how_record_collectors_find_lost_music_and_preserve_our_cultural_heritage': ['2016-11-12',
  752230,
  ['TEDx', 'art', 'culture', 'music']],
 '/talks/alvin_irby_how_to_inspire_every_child_to_be_a_lifelong_reader': ['2017-11-28',
  774478,
  ['TED Residency', 'children', 'education', 'identity']],
 '/talks/amar_inamdar_the_thrilling_potential_for_off_grid_solar_energy

In [466]:
# if I want the key's to be rows, pass orient = 'index'
data_dict = pd.from_dict(my_old_data)

AttributeError: module 'pandas' has no attribute 'from_dict'

In [ ]:
data_dict.describe()

In [ ]:
data_dict.head()

## More Data

There is data from a kaggle that incorporates similar things above, and I will use it to enrich my data.  I will also substitute using it now because I am still troubleshooting pulling data from ted.com